# Задание №1: Анализ лог-данных производственных сенсоров

**Описание данных**: Вам предоставлены сырые текстовые логи, поступающими от производственных сенсоров. Каждая строка RDD представляет собой запись лога, содержащую несколько полей, разделённых символом | . Данные могут быть неполными или содержать ошибки, требующие обработки.

In [ ]:
from pyspark import SparkConf, SparkContext
from datetime import datetime

# Создаем объект SparkConf для настройки конфигурации Spark
conf = (SparkConf()
        .setAppName("sensor_analysis")
        .setMaster("local[*]")
        )

# Создаем объект SparkContext с использованием настроек SparkConf
# sc = SparkContext("local", "sensor_analysis")
sc = SparkContext(conf=conf)

In [ ]:
# timestamp|sensor_id|temperature|pressure|status|error_code

raw_data = """
2025-01-10 08:00:05|S101|25.3|10.1|OK|NULL
2025-01-10 08:05:10|S102|26.1|NULL|WARNING|E01
2025-01-10 08:10:00|S101|25.5|10.2|OK|NULL
2025-01-10 08:15:30|S103|NULL|9.9|ERROR|E05
2025-01-10 08:20:15|S102|25.8|10.0|OK|NULL
2025-01-10 08:25:00|S101|25.4|10.1|OK|NULL
2025-01-10 08:30:20|S104|27.0|10.3|OK|NULL
2025-01-10 08:35:05|S101|25.6|NULL|OK|NULL
2025-01-10 08:40:00|S103|24.9|9.8|WARNING|NULL
2025-01-10 08:45:10|S102|26.0|10.1|OK|NULL
2025-01-11 09:00:00|S104|27.2|10.4|ERROR|E03
2025-01-11 09:05:30|S101|25.7|10.2|OK|NULL
2025-01-11 09:10:00|S105|24.0|9.5|OK|NULL
2025-01-11 09:15:10|S101|25.8|10.3|OK|NULL
2025-01-11 09:20:20|S102|26.2|NULL|WARNING|E02
2025-01-11 09:25:00|S103|25.0|9.7|OK|NULL
2025-01-12 10:00:15|S104|NULL|10.5|ERROR|E04
2025-01-12 10:05:00|S105|24.1|9.6|OK|NULL
2025-01-12 10:10:05|S101|25.9|10.4|OK|NULL
2025-01-12 10:15:00|S102|26.3|10.2|OK|NULL
2025-01-12 10:20:10|S103|NULL|NULL|ERROR|E05
2025-01-13 11:00:00|S104|27.3|10.6|OK|NULL
2025-01-13 11:05:15|S105|24.2|9.7|WARNING|E01
2025-01-13 11:10:00|S101|26.0|10.5|OK|NULL
2025-01-13 11:15:20|S102|NULL|10.3|OK|NULL
2025-01-13 11:20:00|S103|25.1|9.9|OK|NULL
2025-01-14 08:30:10|S104|27.4|NULL|ERROR|E03
2025-01-14 08:35:00|S105|24.3|9.8|OK|NULL
2025-01-14 08:40:15|S101|26.1|10.6|OK|NULL
2025-01-14 08:45:00|S102|26.4|10.4|OK|NULL
2025-01-14 08:50:10|S103|25.2|10.0|WARNING|NULL
2025-01-15 09:00:00|S104|27.5|10.7|OK|NULL
2025-01-15 09:05:15|S105|NULL|9.9|ERROR|E02
2025-01-15 09:10:00|S101|26.2|10.7|OK|NULL
2025-01-15 09:15:20|S102|26.5|NULL|OK|NULL
2025-01-15 09:20:00|S103|25.3|10.1|OK|NULL
2025-01-16 10:00:10|S104|27.6|10.8|OK|NULL
2025-01-16 10:05:00|S105|24.5|10.0|WARNING|E04
2025-01-16 10:10:15|S101|26.3|10.8|OK|NULL
2025-01-16 10:15:00|S102|26.6|10.5|OK|NULL
"""

## 1.1

*   Разделить текст на строки, используя разделитель **'\n'**;
*   Разделить каждую строку RDD на отдельные поля, используя символ **|**  как разделитель.

In [ ]:
# Создаем RDD из списка строк
raw_rdd = sc.parallelize([line for line in raw_data.split('\n') if line.strip()])

# Разделяем каждую строку RDD на отдельные поля
parsed_rdd = raw_rdd.map(lambda line: line.split('|'))
parsed_rdd.collect()

[['2025-01-10 08:00:05', 'S101', '25.3', '10.1', 'OK', 'NULL'],
 ['2025-01-10 08:05:10', 'S102', '26.1', 'NULL', 'WARNING', 'E01'],
 ['2025-01-10 08:10:00', 'S101', '25.5', '10.2', 'OK', 'NULL'],
 ['2025-01-10 08:15:30', 'S103', 'NULL', '9.9', 'ERROR', 'E05'],
 ['2025-01-10 08:20:15', 'S102', '25.8', '10.0', 'OK', 'NULL'],
 ['2025-01-10 08:25:00', 'S101', '25.4', '10.1', 'OK', 'NULL'],
 ['2025-01-10 08:30:20', 'S104', '27.0', '10.3', 'OK', 'NULL'],
 ['2025-01-10 08:35:05', 'S101', '25.6', 'NULL', 'OK', 'NULL'],
 ['2025-01-10 08:40:00', 'S103', '24.9', '9.8', 'WARNING', 'NULL'],
 ['2025-01-10 08:45:10', 'S102', '26.0', '10.1', 'OK', 'NULL'],
 ['2025-01-11 09:00:00', 'S104', '27.2', '10.4', 'ERROR', 'E03'],
 ['2025-01-11 09:05:30', 'S101', '25.7', '10.2', 'OK', 'NULL'],
 ['2025-01-11 09:10:00', 'S105', '24.0', '9.5', 'OK', 'NULL'],
 ['2025-01-11 09:15:10', 'S101', '25.8', '10.3', 'OK', 'NULL'],
 ['2025-01-11 09:20:20', 'S102', '26.2', 'NULL', 'WARNING', 'E02'],
 ['2025-01-11 09:25:00', '

## 1.2

*  Преобразовать поля (temperature, pressure) в числовой тип (float);
*  Преобразовать NULL значения в None;

In [ ]:
transformed_rdd = parsed_rdd.map(
    lambda p: (
        datetime.strptime(p[0], "%Y-%m-%d %H:%M:%S"),  # Timestamp
        p[1],                                          # Sensor ID
        float(p[2]) if p[2] != "NULL" else None,       # Temperature
        float(p[3]) if p[3] != "NULL" else None,       # Pressure
        p[4],                                          # Status
        p[5] if p[5] != "NULL" else None               # Error code
    )
)

## 1.3 Подсчет количества записей для каждого status

In [ ]:
# Извлекаем статус и создаем пару (status, 1)

# Статус находится по индексу 4
status_counts_rdd = transformed_rdd.map(lambda r: (r[4], 1))
status_counts_rdd.collect()

[('OK', 1),
 ('WARNING', 1),
 ('OK', 1),
 ('ERROR', 1),
 ('OK', 1),
 ('OK', 1),
 ('OK', 1),
 ('OK', 1),
 ('WARNING', 1),
 ('OK', 1),
 ('ERROR', 1),
 ('OK', 1),
 ('OK', 1),
 ('OK', 1),
 ('WARNING', 1),
 ('OK', 1),
 ('ERROR', 1),
 ('OK', 1),
 ('OK', 1),
 ('OK', 1),
 ('ERROR', 1),
 ('OK', 1),
 ('WARNING', 1),
 ('OK', 1),
 ('OK', 1),
 ('OK', 1),
 ('ERROR', 1),
 ('OK', 1),
 ('OK', 1),
 ('OK', 1),
 ('WARNING', 1),
 ('OK', 1),
 ('ERROR', 1),
 ('OK', 1),
 ('OK', 1),
 ('OK', 1),
 ('OK', 1),
 ('WARNING', 1),
 ('OK', 1),
 ('OK', 1)]

In [ ]:
# Суммируем количество для каждого статуса
total_status_counts = status_counts_rdd.reduceByKey(lambda a, b: a + b)

for status, count in total_status_counts.collect():
    print(f"{status}: {count}")

OK: 28
ERROR: 6


## 1.4 Среднее значение температуры:
*   Исключите записи без температуры из расчета среднего;
*   Округлите до двух знаков после запятой.

In [ ]:
# Расчет среднего значения температуры

avg_temperature = (transformed_rdd
                  .filter(lambda record: record[2] is not None)   # Отфильтровываем записи без температуры
                  .map(lambda record: (record[2], 1))             # Создаем пары (температура, 1)
)
avg_temperature.collect()

[(25.3, 1),
 (26.1, 1),
 (25.5, 1),
 (25.8, 1),
 (25.4, 1),
 (27.0, 1),
 (25.6, 1),
 (24.9, 1),
 (26.0, 1),
 (27.2, 1),
 (25.7, 1),
 (24.0, 1),
 (25.8, 1),
 (26.2, 1),
 (25.0, 1),
 (24.1, 1),
 (25.9, 1),
 (26.3, 1),
 (27.3, 1),
 (24.2, 1),
 (26.0, 1),
 (25.1, 1),
 (27.4, 1),
 (24.3, 1),
 (26.1, 1),
 (26.4, 1),
 (25.2, 1),
 (27.5, 1),
 (26.2, 1),
 (26.5, 1),
 (25.3, 1),
 (27.6, 1),
 (24.5, 1),
 (26.3, 1),
 (26.6, 1)]

In [ ]:
# Суммируем все температуры и подсчитываем количество показаний
sum_and_count = avg_temperature.reduce(lambda a, b: (a[0] + b[0], a[1] + b[1]))

# Вычисляем среднее значение
average_temp = sum_and_count[0] / sum_and_count[1]
print(f"Средняя температура: {average_temp:.2f}°C")

Средняя температура: 25.84°C


## 1.5 Количество ошибок от сенсоров

In [ ]:
# Подсчет активных сенсоров с ошибками
active_sensors_with_errors = (
    transformed_rdd
    .filter(lambda record: record[4] == "ERROR")  # Фильтруем только записи с ошибками
    .map(lambda record: (record[1], 1))           # Создаем пары (sensor_id, 1)
)

# Группируем по sensor_id и подсчитываем количество ошибок
sensor_error_counts = (
    active_sensors_with_errors
    .reduceByKey(lambda a, b: a + b)  # Суммируем количество ошибок
    .sortByKey())                     # Сортируем по имени датчика

print("Количество ошибок по датчикам:")
if sensor_error_counts.isEmpty():
    print("Нет датчиков с ошибками")
else:
    for sensor, error_count in sensor_error_counts.collect():
        print(f"Датчик {sensor}: {error_count} ошибок")

Количество ошибок по датчикам:
Датчик S103: 2 ошибок
Датчик S104: 3 ошибок
Датчик S105: 1 ошибок


## 1.5 Количество ошибок для каждого кода

In [ ]:
# Подсчет сенсоров с кодом ошибок
sensors_with_error_code = (
    transformed_rdd
    .filter(lambda record: record[5] is not None)   # Фильтруем записи с ошибками
    .map(lambda record: (record[5], 1))             # Создаем пары (error_code, 1)
)

# Группируем по error_code и подсчитываем количество ошибок
error_code_counts = (
    sensors_with_error_code
    .reduceByKey(lambda a, b: a + b)  # Суммируем количество ошибок
    .sortByKey())                     # Сортируем по имени датчика

print("Количество ошибок по коду:")
if error_code_counts.isEmpty():
    print("Нет датчиков с ошибками")
else:
    for error, error_count in error_code_counts.collect():
        print(f"Код ошибки {error}: {error_count} ошибок")

Количество ошибок по коду:
Код ошибки E01: 2 ошибок
Код ошибки E02: 2 ошибок
Код ошибки E03: 2 ошибок
Код ошибки E04: 2 ошибок
Код ошибки E05: 2 ошибок


## 1.6 Записи с высоким давлением и температурой
 Отфильтровать RDD, чтобы получить только те записи, где

*   pressure выше **10.0**
*   temperature выше **26.0**

Для каждой отфильтрованной записи выведите кортеж (timestamp, sensor_id, temperature, pressure)

In [ ]:
# Фильтрация показаний датчиков
critical_rdd = (
    transformed_rdd
    .filter(lambda r: r[2] is not None and r[3] is not None)  # Отсеиваем записи с None
    .filter(lambda r: r[2] > 26 and r[3] > 10)                # Температура >26 и давление >10
    .sortBy(lambda r: r[0])                                   # Сортировка по дате
)

# Форматированный вывод
print("\n" + "="*65)
print("КРИТИЧЕСКИЕ ПОКАЗАНИЯ ДАТЧИКОВ".center(65))
print(f"{'Температура >26°C и давление >10':^65}")
print("="*65)
print("{:<20} {:<6} {:<12} {:<10} {:<10}".format("Время", "Датчик", "Температура", "Давление", "Статус"))
print("-"*65)

if critical_rdd.isEmpty():
    print("Нет критических показаний".center(65))
else:
    for record in critical_rdd.collect():
        print("{:<20} {:<6} {:>8.1f}°C {:>8.1f} {:>10}".format(
            record[0].strftime("%Y-%m-%d %H:%M:%S"),
            record[1],
            record[2],
            record[3],
            record[4]
        ))


                  КРИТИЧЕСКИЕ ПОКАЗАНИЯ ДАТЧИКОВ                 
                Температура >26°C и давление >10                 
Время                Датчик Температура  Давление   Статус    
-----------------------------------------------------------------
2025-01-10 08:30:20  S104       27.0°C     10.3         OK
2025-01-11 09:00:00  S104       27.2°C     10.4      ERROR
2025-01-12 10:15:00  S102       26.3°C     10.2         OK
2025-01-13 11:00:00  S104       27.3°C     10.6         OK
2025-01-14 08:40:15  S101       26.1°C     10.6         OK
2025-01-14 08:45:00  S102       26.4°C     10.4         OK
2025-01-15 09:00:00  S104       27.5°C     10.7         OK
2025-01-15 09:10:00  S101       26.2°C     10.7         OK
2025-01-16 10:00:10  S104       27.6°C     10.8         OK
2025-01-16 10:10:15  S101       26.3°C     10.8         OK
2025-01-16 10:15:00  S102       26.6°C     10.5         OK


In [ ]:
# Остановка SparkContext
sc.stop()

# Задание №2: Анализ паролей

In [ ]:
import requests

# Получение "прямой" ссылки
public_url = "https://disk.yandex.ru/d/Vgm0jVCat6ZYiQ"
api_response = requests.get(f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={public_url}")
api_response.raise_for_status()

file_response = requests.get(api_response.json()['href'])
file_response.raise_for_status()

# Сохранение в файл
with open("password.txt", "wb") as f:
    f.write(file_response.content)

## 2.1 Загрузка и предварительная обработка


*   загрузить файл в RDD;
*   очистить от лишних пробелов;
*   удалить пустые строки;

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "password_analysis")

password_rdd = (sc.textFile("password.txt")
                .map(lambda x: x.strip())          # удаление пробелов
                .filter(lambda x: x)               # удаление пустых строк
)

## 2.2 Длина паролей:
*   Рассчитать среднюю длину пароля во всем списке. Округлить до целого числа.
*   Найдите минимальную и максимальную длину паролей.
*   Найдите минимальную и максимальную длину паролей.

In [ ]:
length_stats = password_rdd.map(lambda x: len(x)).stats()
length_stats

(count: 1000000, mean: 9.363507000000327, stdev: 2.833534834963496, max: 15.0, min: 6.0)

In [ ]:
# Средняя длина пароля (округленная)
avg_length = length_stats.mean()
print(f"Средняя длина пароля: {round(avg_length)} символов")

Средняя длина пароля: 9 символов


In [ ]:
# Минимальная и максимальная длина
print(f"Минимальная длина пароля: {length_stats.min()}")
print(f"Максимальная длина пароля: {length_stats.max()}")

Минимальная длина пароля: 6.0
Максимальная длина пароля: 15.0


## 2.3 Символьный состав:
*   Количество паролей, содержащих только цифры
*   Количество паролей, содержащих только буквы (как строчные, так и заглавные)

In [ ]:
# Количество паролей, содержащих только цифры
only_digit = password_rdd.filter(lambda x: x.isdigit())
print(f"Количество паролей, содержащих только цифры: {only_digit.count()}")

# Количество паролей, содержащих только буквы
only_alpha = password_rdd.filter(lambda x: x.isalpha())
print(f"Количество паролей, содержащих только буквы: {only_alpha.count()}")

Количество паролей, содержащих только цифры: 441347
Количество паролей, содержащих только буквы: 115088


## 2.5 Топ длин


In [ ]:
len_password = password_rdd.map(lambda x: len(x)) # Получаем длину каждого пароля

top_len = (len_password
    .map(lambda x: (x, 1))                   # Создаем пары (длина, 1)
    .reduceByKey(lambda a, b: a + b)         # Считаем количество для каждой длины
    .top(5, key=lambda x: x[1])              # Берем топ-5 по количеству
)

print("Топ-5 самых распространенных длин паролей:")
for num, cnt in top_len:
    print(f"{num}: {cnt}")

Топ-5 самых распространенных длин паролей:
6: 215887
8: 122106
7: 115450
10: 113429
9: 110343


## Топ префиксов/суффиксов
*   топ-5 самых распространенных префикса длиной 3 символа
*   топ-5 самых распространенных суффикса длиной 3 символа

In [ ]:
# Топ-5 самых распространенных префикса
list_perfix = password_rdd.map(lambda x: (x[0:3], 1))
cnt_perfix = list_perfix.reduceByKey(lambda a, b: a + b)
top_perfix = cnt_perfix.top(5, key = lambda x: x[1])

print("Топ-5 самых распространенных префикса")
for perfix, cnt in top_perfix:
    print(f"{perfix}: {cnt}")

Топ-5 самых распространенных префикса
123: 17857
qwe: 17054
pas: 13290
654: 9925
edc: 9813


In [ ]:
# Топ-5 самых распространенных суффикса
list_suffix = password_rdd.map(lambda x: (x[-3:], 1))
cnt_suffix = list_suffix.reduceByKey(lambda a, b: a + b)
top_suffix = cnt_suffix.top(5, key = lambda x: x[1])

print("Топ-5 самых распространенных суффикса")
for suffix, cnt in top_suffix:
    print(f"{suffix}: {cnt}")

Топ-5 самых распространенных суффикса
123: 16135
789: 15266
020: 14636
021: 14558
022: 14552


In [ ]:
# Остановка SparkContext
sc.stop()

# Задание №3: Анализ словаря русских слов

In [ ]:
import requests

# Получение "прямой" ссылки
public_url = "https://disk.yandex.ru/d/sF4QUi3TBAXspw"
api_response = requests.get(f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={public_url}")
api_response.raise_for_status()

file_response = requests.get(api_response.json()['href'])
file_response.raise_for_status()

# Сохранение в файл
with open("Rus_dict.txt", "wb") as f:
    f.write(file_response.content)

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "dict_analysis")

dict_rdd = (sc.textFile("Rus_dict.txt")
                .map(lambda x: x.strip().lower())   # удаление пробелов и приведение к нижнему регистру
                .filter(lambda x: x)                # удаление пустых строк
)

## 3.1 Статистика по словам:

*   Общее количество слов в словаре
*   Топ-5 самых длинных слов
*   Топ-5 самых коротких слов



In [ ]:
# Количество слов
print(f"Количество слов в словаре: {dict_rdd.count()}")

Количество слов в словаре: 51301


In [ ]:
# Топ-5 самых длинных слов
print("Топ-5 самых длинных слов:")
longest_words = (dict_rdd.map(lambda x: (len(x), x))  # Пары (длина, слово)
                     .sortByKey(ascending=False)      # Сортировка по убыванию длины
                     .take(5))                        # Получаем список

for length, word in longest_words:
    print(f"{word} ({length} букв)")

Топ-5 самых длинных слов:
человеконенавистничество (24 букв)
интернационализирование (23 букв)
неусовершенствованность (23 букв)
переосвидетельствование (23 букв)
адмиралтейств-коллегия (22 букв)


In [ ]:
# Топ-5 самых коротких слов
print("Топ-5 самых коротких слов:")
shortest_words = (dict_rdd
                  .map(lambda x: (len(x), x))           # Пары (длина, слово)
                  #.sortByKey()                         # Сортировка по возрастанию длины
                  #.take(5)
                  .top(5, key=lambda x: x[1])
)

for length, word in shortest_words:
    print(f"{word} ({length} букв)")

Топ-5 самых коротких слов:
ёршик (5 букв)
ёрш (3 букв)
ёрничество (10 букв)
ёрник (5 букв)
ёрзание (7 букв)


## 3.2 Анализ по длине слов

In [ ]:
# Распространение по длине слов
lengths_counts = (dict_rdd
              .map(lambda word: (len(word), 1))       # Пары (длина, 1)
              .reduceByKey(lambda a, b: a + b)        # Суммируем по длинам
              .sortByKey()                            # Сортировка по длине слова
)

print("Распространение по длине слов:")
for length, cnt in lengths_counts.collect():
    print(f"Слов длиной {length} букв: {cnt}")

Топ-5 самых распространенных длин слов:
Слов длиной 2 букв: 54
Слов длиной 3 букв: 472
Слов длиной 4 букв: 1607
Слов длиной 5 букв: 3483
Слов длиной 6 букв: 4836
Слов длиной 7 букв: 6335
Слов длиной 8 букв: 6885
Слов длиной 9 букв: 6612
Слов длиной 10 букв: 5501
Слов длиной 11 букв: 4591
Слов длиной 12 букв: 3527
Слов длиной 13 букв: 2774
Слов длиной 14 букв: 1994
Слов длиной 15 букв: 1123
Слов длиной 16 букв: 692
Слов длиной 17 букв: 364
Слов длиной 18 букв: 227
Слов длиной 19 букв: 117
Слов длиной 20 букв: 60
Слов длиной 21 букв: 29
Слов длиной 22 букв: 14
Слов длиной 23 букв: 3
Слов длиной 24 букв: 1


In [ ]:
# Часто встречающаяся длина слова
most_common_length = lengths_counts.top(1, key=lambda x: x[1])

print(f"Наиболее частая длина слов {most_common_length[0]} букв")

Наиболее частая длина слов 8 букв


## 3.2 Символьный анализ

In [ ]:
# Количество слов содержащих букву 'ё'

words_with_letter = dict_rdd.filter(lambda word: 'ё' in word)
letter_count = words_with_letter.count()
print(f"Слов с буквой 'ё': {letter_count}")

Слов с буквой 'ё': 2111


In [ ]:
# Поиск палиндромов

def is_palindrome(word):
    return word == word[::-1]

palindromes = dict_rdd.filter(is_palindrome)
palindrome_list = palindromes.collect()  # Собираем все палиндромы

print("Первые 10 элементов списка")
for word in palindrome_list[:10]:
    print(word)

Первые 10 элементов списка
ага
боб
дед
довод
доход
заказ
кабак
казак
киник
кок


In [ ]:
sc.stop()

# Задание №4: Анализ логов веб-сервера

In [ ]:
import requests
import re

# Получение "прямой" ссылки
public_url = "https://disk.yandex.ru/d/tbZFWvv80u8BPA"
api_response = requests.get(f"https://cloud-api.yandex.net/v1/disk/public/resources/download?public_key={public_url}")
api_response.raise_for_status()

file_response = requests.get(api_response.json()['href'])
file_response.raise_for_status()

# Сохранение в файл
with open("logfiles.log", "wb") as f:
    f.write(file_response.content)

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "log_analysis")

logs_rdd = sc.textFile("logfiles.log")

## 4.1 Предварительная обработка
Написать функцию **parse_log**, которая будет извлекать поля в кортеже:

*   IP-адрес
*   Запрос
*   Статус-код ответа
*   Размер ответа в байтах

In [ ]:
def parse_log(log_line):
    pattern = r'(\S+) - - \[(.*?)\] "(.*?)" (\d{3}) (\d+)'
    match = re.match(pattern, log_line)
    if match:
        ip = match.group(1)
        request = match.group(3)
        status_code = int(match.group(4))
        response_size = int(match.group(5))
        return (ip, request, status_code, response_size)
    return None

In [ ]:
# Применение функции для парсинга

parsed_logs_rdd = (logs_rdd.map(parse_log)
                        .filter(lambda x: x is not None)
)

## 4.2 Общая статистика по логам

*   Общее количество запросов
*   Средний размер ответа сервера (в байтах) по всем запросам
*   Количество уникальных IP-адресов



In [ ]:
# Общее количество запросов
print(f"Общее количество запросов: {parsed_logs_rdd.count()}")

Общее количество запросов: 10000


In [ ]:
# Расчет среднего размера ответа
avg_size = parsed_logs_rdd.map(lambda x: x[3]).mean()

print(f"Средний размер ответа сервера: {round(avg_size)} байт")

Средний размер ответа сервера: 4999 байт


In [ ]:
# Количество уникальных IP-адресов
unique_ips = parsed_logs_rdd.map(lambda x: x[0]).distinct().count()

print(f"Количество уникальных IP-адресов: {unique_ips}")

Количество уникальных IP-адресов: 10000


## 4.3 Анализ HTTP-статусов

In [ ]:
# Количество запросов по статус-кодам

status_codes = (parsed_logs_rdd
                  .map(lambda x: (x[2], 1))
                  .reduceByKey(lambda a, b: a + b)
                  .sortByKey()
)

print("Распределение по HTTP-кодам:")
for code, cnt in status_codes.collect():
    print(f"{code}: {cnt} запросов")

Распределение по HTTP-кодам:
200: 3850 запросов
301: 780 запросов
302: 773 запросов
400: 734 запросов
401: 781 запросов
403: 735 запросов
404: 820 запросов
500: 797 запросов
502: 730 запросов


In [ ]:
# Доля успешных запросов (статус-код 200)

success_count = parsed_logs_rdd.filter(lambda x: x[2] == 200).count()
total_count = parsed_logs_rdd.count()

# Расчёт доли успешных запросов
success_rate = (success_count / total_count) * 100 if total_count > 0 else 0

print(f"Успешных запросов: {success_count}/{total_count} ({success_rate:.1f}%)")

Успешных запросов: 3850/10000 (38.5%)


## 4.4 Анализ эндпоинтов и запросов

In [ ]:
# Топ-5
endpoint_counts = (parsed_logs_rdd
                     .map(lambda x: x[1].split()[1] if len(x[1].split()) > 1 else None)  # Безопасное извлечение
                     .filter(lambda x: x is not None)         # Игнорируем некорректные записи
                     .map(lambda endpoint: (endpoint, 1))
                     .reduceByKey(lambda a, b: a + b)
                     #.sortByKey()
                     #.take(5)
                     .top(5, key=lambda x: x[1])              # Берем топ-5 по количеству
)

print("\nТоп-5 самых популярных эндпоинтов:")
for i, (endpoint, count) in enumerate(endpoint_counts, 1):
    print(f"{i}. {endpoint}: {count} запросов")


Топ-5 самых популярных эндпоинтов:
1. /search?q=spark: 952 запросов
2. /docs/api: 949 запросов
3. /admin/dashboard: 935 запросов
4. /auth/register: 929 запросов
5. /: 913 запросов


In [ ]:
# Количество запросов по типам (GET, POST, PUT и т.д.)

endpoint_counts = (parsed_logs_rdd
                     .map(lambda x: x[1].split()[0] if len(x[1].split()) > 1 else None)  # Безопасное извлечение
                     .filter(lambda x: x is not None)  # Игнорируем некорректные записи
                     .map(lambda endpoint: (endpoint, 1))
                     .reduceByKey(lambda a, b: a + b)
                     .sortByKey()
)

print("Распределение по HTTP-кодам:")
for code, cnt in endpoint_counts.collect():
    print(f"{code}: {cnt} запросов")

Распределение по HTTP-кодам:
DELETE: 1711 запросов
GET: 1677 запросов
HEAD: 1669 запросов
OPTIONS: 1662 запросов
POST: 1641 запросов
PUT: 1640 запросов


In [ ]:
sc.stop()